In [30]:
import sklearn
import pandas as pd
import numpy as np
import pickle
import subprocess
import shlex
from collections import defaultdict
import copy
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_recall_fscore_support, confusion_matrix
from statsmodels.stats.weightstats import ztest
# from statsmodels.stats.weightstats.CompareMeans import ztest_ind

In [31]:
def pkl_process(data=None, file=None, mode="dump"):
    f_mode = "wb" if mode == "dump" else "rb"
    load_data = None
    
    with open(file, f_mode) as f:
        if mode == "dump":
            pickle.dump(data, f)
        else:
            load_data = pickle.load(f)
    
    return load_data

def isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False
    return True

def best_full_cv_auc(model, tuned_parameters, train_X, train_y, score='roc_auc', seed=13):
    gs = GridSearchCV(model, tuned_parameters, scoring=score, n_jobs=-1, cv=5, verbose=0, iid=True) #scoring='roc_auc'
    l = list(zip(list(train_X), list(train_y)))
    np.random.seed(seed)
    np.random.shuffle(l)
    X = np.array(list(map(lambda x: x[0], l)))
    y = np.array(list(map(lambda x: x[1], l)))
    gs.fit(X, y)
    return gs.best_estimator_, gs.best_score_

def predict_auc_roc(clf, test_X, test_y):
    pred_result = clf.predict_proba(test_X)
    idx = list(filter(lambda x: x==1, clf.classes_))[0]
    pred_result_1 = list(map(lambda x: x[idx], pred_result))
    return roc_auc_score(test_y, pred_result_1)

#used for two models on same data
def performance_diff_by_p_value(model1, model2, train_X, train_y, seeds):
    l = list(zip(list(train_X), list(train_y)))
    auc_m1 = []
    auc_m2 = []
    
    for seed in seeds:
        np.random.seed(seed)
        np.random.shuffle(l)
        nl = l[:100]
        X = np.array(list(map(lambda x: x[0], nl)))
        y = np.array(list(map(lambda x: x[1], nl)))
        
        auc_m1.append(predict_auc_roc(model1, X, y))
        auc_m2.append(predict_auc_roc(model2, X, y))
    
    return (np.mean(auc_m1), np.std(auc_m1)), (np.mean(auc_m2), np.std(auc_m2)), ztest(auc_m1, auc_m2)

#used for two models on different data
def performance_diff_by_p_value_mix_ohe(model1, model2, train_X1, train_y1, train_X2, train_y2, seeds):
    l1 = list(zip(list(train_X1), list(train_y1)))
    l2 = list(zip(list(train_X2), list(train_y2)))
    auc_m1 = []
    auc_m2 = []
    
    for seed in seeds:
        np.random.seed(seed)
        
        np.random.shuffle(l1)
        nl1 = l1[:100]
        X1 = np.array(list(map(lambda x: x[0], nl1)))
        y1 = np.array(list(map(lambda x: x[1], nl1)))
        
        np.random.shuffle(l1)
        nl2 = l2[:100]
        X2 = np.array(list(map(lambda x: x[0], nl2)))
        y2 = np.array(list(map(lambda x: x[1], nl2)))
        
        auc_m1.append(predict_auc_roc(model1, X1, y1))
        auc_m2.append(predict_auc_roc(model2, X2, y2))
    
    return (np.mean(auc_m1), np.std(auc_m1)), (np.mean(auc_m2), np.std(auc_m2)), ztest(auc_m1, auc_m2)
    

In [32]:
seeds = [p for p in range(100000) if isPrime(p)][100:200]
seed = 13

In [22]:
'''
wh = with homa
wth = without homa
ohe = one hot encoded
mix = one hot encoded + real value
'''

'\nwh = with homa\nwth = without homa\nohe = one hot encoded\nmix = one hot encoded + real value\n'

In [23]:
data_wh_ohe_X = np.array(list(pkl_process(file="data/train_ohe_x.pkl",mode='load')) + list(pkl_process(file="data/test_ohe_x.pkl",mode='load')))

data_wh_ohe_X.shape

(488, 73)

In [24]:
data_wh_ohe_y_nafld = np.array(list(pkl_process(file="data/train_ohe_y_nafld.pkl",mode='load')) + list(pkl_process(file="data/test_ohe_y_nafld.pkl",mode='load')))

data_wh_ohe_y_nash = np.array(list(pkl_process(file="data/train_ohe_y_nash.pkl",mode='load')) + list(pkl_process(file="data/test_ohe_y_nash.pkl",mode='load')))

data_wh_ohe_y_fib = np.array(list(pkl_process(file="data/train_ohe_y_fib.pkl",mode='load')) + list(pkl_process(file="data/test_ohe_y_fib.pkl",mode='load')))

data_wh_ohe_y_nafld.shape, data_wh_ohe_y_nash.shape, data_wh_ohe_y_fib.shape

((488,), (488,), (488,))

In [25]:
data_wh_mix_X_nafld = np.array(list(pkl_process(file="data/MIX_train_x_nalfd.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_x_nalfd.pkl",mode='load')))

data_wh_mix_X_nash = np.array(list(pkl_process(file="data/MIX_train_x_nash.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_x_nash.pkl",mode='load')))

data_wh_mix_X_fib = np.array(list(pkl_process(file="data/MIX_train_x_fib.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_x_fib.pkl",mode='load')))

data_wh_mix_X_nafld.shape, data_wh_mix_X_nash.shape, data_wh_mix_X_fib.shape

((488, 39), (488, 39), (488, 39))

In [26]:
data_wh_mix_y_nafld = np.array(list(pkl_process(file="data/MIX_train_y_nalfd.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_y_nalfd.pkl",mode='load')))

data_wh_mix_y_nash = np.array(list(pkl_process(file="data/MIX_train_y_nash.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_y_nash.pkl",mode='load')))

data_wh_mix_y_fib = np.array(list(pkl_process(file="data/MIX_train_y_fib.pkl",mode='load')) + list(pkl_process(file="data/MIX_test_y_fib.pkl",mode='load')))

data_wh_mix_y_nafld.shape, data_wh_mix_y_nash.shape, data_wh_mix_y_fib.shape

((488,), (488,), (488,))

In [33]:
lr_tuned_parameters = {'max_iter':[100, 500, 1000], 'tol':[0.00001, 0.0001, 0.001, 0.01, 0.1],'random_state':[seed], 'C': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 
                    'solver':['lbfgs', 'liblinear', 'newton-cg'], 'class_weight': [None, 'balanced']}

lr_nafld_clf, lr_nafld_auc = best_full_cv_auc(LogisticRegression(), lr_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nafld)
lr_nash_clf, lr_nash_auc = best_full_cv_auc(LogisticRegression(), lr_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nash)
lr_fib_clf, lr_fib_auc = best_full_cv_auc(LogisticRegression(), lr_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_fib)

'AUC_ROC nafld: ', lr_nafld_auc, " nash: ", lr_nash_auc, " fib: ", lr_fib_auc

('AUC_ROC nafld: ',
 0.8751285549366107,
 ' nash: ',
 0.7986681233784118,
 ' fib: ',
 0.8027065906423568)

In [43]:
svm_tuned_parameters = {"C": [0.01, 0.1, 2, 64, 128, 512, 1024, 2048], 'probability':[True], 'tol': [0.1, 0.01, 0.001, 0.0001], 
                    'random_state':[seed], 'gamma':['auto', 'scale']}

svm_nafld_clf, svm_nafld_auc = best_full_cv_auc(SVC(), svm_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nafld)
svm_nash_clf, svm_nash_auc = best_full_cv_auc(SVC(), svm_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nash)
svm_fib_clf, svm_fib_auc = best_full_cv_auc(SVC(), svm_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_fib)

'AUC_ROC nafld: ', svm_nafld_auc, " nash: ", svm_nash_auc, " fib: ", svm_fib_auc

('AUC_ROC nafld: ',
 0.8778820028269375,
 ' nash: ',
 0.8009633502920671,
 ' fib: ',
 0.7618325914048425)

In [44]:
#tstat, pvalue
performance_diff_by_p_value(lr_nafld_clf, svm_nafld_clf, data_wh_ohe_X, data_wh_ohe_y_nafld, seeds)

((0.9096500970818727, 0.024858190544789997),
 (0.9247184088871439, 0.022517919071519933),
 (-4.4700108332897965, 7.821563540513267e-06))

In [45]:
performance_diff_by_p_value(lr_nash_clf, svm_nash_clf, data_wh_ohe_X, data_wh_ohe_y_nash, seeds)

((0.8413878242786926, 0.03772028658905607),
 (0.9145507586153206, 0.026688783521812153),
 (-15.75427654662244, 6.418972070324991e-56))

In [46]:
performance_diff_by_p_value(lr_fib_clf, svm_fib_clf, data_wh_ohe_X, data_wh_ohe_y_fib, seeds)

((0.8301812543673626, 0.057582875355909446),
 (0.946557078277265, 0.041084568257042105),
 (-16.369426471167095, 3.161794168747819e-60))

In [38]:
RF_tuned_parameters = {'n_estimators':[10, 50, 100, 500, 1000], 'criterion':['gini', 'entropy'], 'random_state':[seed], 
                    'max_features':['log2', 'auto', None], 'min_samples_split':[2, 4], 'max_depth': [2, 5, 10, 25, 50, None], 
                    'min_samples_leaf':[1,2,4], 'class_weight': [None, 'balanced', 'balanced_subsample']}

# svm_ohe_nafld_clf, svm_nafld_auc = best_full_cv_auc(SVC(), RF_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nafld)
# svm_ohe_nash_clf, svm_nash_auc = best_full_cv_auc(SVC(), RF_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_nash)
# svm_ohe_fib_clf, svm_fib_auc = best_full_cv_auc(SVC(), RF_tuned_parameters, data_wh_ohe_X, data_wh_ohe_y_fib)

# 'AUC_ROC nafld: ', svm_nafld_auc, " nash: ", svm_nash_auc, " fib: ", svm_fib_auc

In [39]:
rf_mix_nafld_clf, rf_mix_nafld_auc = best_full_cv_auc(RandomForestClassifier(), RF_tuned_parameters, data_wh_mix_X_nafld, data_wh_mix_y_nafld)

rf_mix_nash_clf, rf_mix_nash_auc = best_full_cv_auc(RandomForestClassifier(), RF_tuned_parameters, data_wh_mix_X_nash, data_wh_mix_y_nash)

rf_mix_fib_clf, rf_mix_fib_auc = best_full_cv_auc(RandomForestClassifier(), RF_tuned_parameters, data_wh_mix_X_fib, data_wh_mix_y_fib)

'AUC_ROC nafld: ', rf_mix_nafld_auc, " nash: ", rf_mix_nash_auc, " fib: ", rf_mix_fib_auc

('AUC_ROC nafld: ',
 0.9131812047206869,
 ' nash: ',
 0.8244170888231799,
 ' fib: ',
 0.8781812052880626)

In [40]:
performance_diff_by_p_value_mix_ohe(lr_nafld_clf, rf_mix_nafld_clf, data_wh_ohe_X, data_wh_ohe_y_nafld, data_wh_mix_X_nafld, data_wh_mix_y_nafld, seeds)

((0.9131376233322199, 0.031118388237403868),
 (1.0, 0.0),
 (-27.773602180906128, 9.04133808955634e-170))

In [41]:
performance_diff_by_p_value_mix_ohe(lr_nafld_clf, rf_mix_nash_clf, data_wh_ohe_X, data_wh_ohe_y_nafld, data_wh_mix_X_nash, data_wh_mix_y_nash, seeds)

((0.9131376233322199, 0.031118388237403868),
 (0.9963869931754316, 0.0),
 (-26.618369984045408, 4.1606231559634274e-156))

In [42]:
performance_diff_by_p_value_mix_ohe(lr_nafld_clf, rf_mix_fib_clf, data_wh_ohe_X, data_wh_ohe_y_nafld, data_wh_mix_X_fib, data_wh_mix_y_fib, seeds)

((0.9131376233322199, 0.031118388237403868),
 (0.9952941176470591, 3.3306690738754696e-16),
 (-26.268931120838214, 4.344630908756933e-152))